In [1]:
import morphio

In [2]:
import os
from os.path import join
data_folder = join(os.path.dirname(morphio.__file__), 'tests/data')
filename = join(data_folder, 'simple.swc')

# Reading a morphology file

In [3]:
cell = morphio.Morphology(filename)
cell

# Accessing arrays of data

In [4]:
cell.points

array([[ 0.,  0.,  0.],
       [ 0.,  5.,  0.],
       [ 0.,  5.,  0.],
       [-5.,  5.,  0.],
       [ 0.,  5.,  0.],
       [ 6.,  5.,  0.],
       [ 0.,  0.,  0.],
       [ 0., -4.,  0.],
       [ 0., -4.,  0.],
       [ 6., -4.,  0.],
       [ 0., -4.,  0.],
       [-5., -4.,  0.]], dtype=float32)

In [5]:
cell.diameters

array([2., 2., 2., 3., 2., 3., 2., 2., 2., 4., 2., 4.], dtype=float32)

In [6]:
cell.perimeters

array([], dtype=float32)

# Accessing morphology sub-regions

In [7]:
cell.soma

In [8]:
cell.sections

In [9]:
cell.section(0)

In [10]:
cell.section(8)

RawDataError: Requested section ID (8) is out of array bounds (array size = 6)

# Accessing data per region

## Section

In [11]:
cell.section(0).points

array([[0., 0., 0.],
       [0., 5., 0.]], dtype=float32)

In [12]:
cell.section(0).diameters

array([2., 2.], dtype=float32)

In [13]:
cell.section(0).perimeters

array([], dtype=float32)

## Root sections (neurite first sections)

In [14]:
# All sections connected to the soma are called 'root sections'
# The soma is not a section
cell.root_sections

[<morphio.Section at 0x7f1d600d0928>, <morphio.Section at 0x7f1d600d08f0>]

## Soma

In [15]:
cell.soma.points

array([[0., 0., 0.]], dtype=float32)

In [16]:
cell.soma.diameters

array([2.], dtype=float32)

In [17]:
# No perimeter

## Mitochondria

In [18]:
cell.mitochondria

# Graph traversal

## Basic properties

In [19]:
# All sections connected to the soma are called 'root sections'
# Nope, the soma is not a section.
cell.section(0).is_root

True

In [20]:
cell.section(1).parent

In [21]:
# Always check if the section is a root section before calling Section.parent
cell.section(0).parent

MissingParentError: Cannot call Section::parent() on a root node (section id=0).

In [22]:
cell.section(0).children

[<morphio.Section at 0x7f1d600d0b20>, <morphio.Section at 0x7f1d600d0b58>]

## Neurite ordering

In [23]:
# Defaults to order of appearance in the file
[section.type for section in cell.root_sections]

[SectionType.basal_dendrite, SectionType.axon]

In [24]:
# Can be changed with the opening flag morphio.nrn_order to respect NEURON simulator ordering
# soma -> axon -> basal dendrite -> apical dendrite
cell2 = morphio.Morphology(filename, options=morphio.nrn_order)
[section.type for section in cell2.root_sections]

[SectionType.axon, SectionType.basal_dendrite]

## Section iteration

In [25]:
from morphio import IterType

IterType.__members__

{u'breadth_first': IterType.breadth_first,
 u'depth_first': IterType.depth_first,
 u'upstream': IterType.upstream}

In [26]:
section = cell.section(0)

breadth_first_iter = section.iter(IterType.breadth_first)
list(breadth_first_iter)

In [27]:
# Upstream iteration last point is always a root section
# Did I tell you the soma is not a section ?
list(cell.section(2).iter(IterType.upstream))[-1].is_root

True

## All neurites section iteration

In [28]:
list(cell.iter())